## Retrieving Volcano Locations
___

We have access to a datafile with 65 different volcano locations and approximate aperture sizes. We will use this to extract location specific data fromt the [MODVOLC](http://modis.higp.hawaii.edu/cgi-bin/modisnew.cgi) database.

In [1]:
import pandas as pd
from os import mkdir
from os.path import isdir

In [2]:
df_loc = pd.read_csv("data/volcanoes.txt", delimiter='\s', header=None, engine='python');

df_loc.columns = ["name", "long", "lat", "aperture"]

df_loc.head()

,name,long,lat,aperture
0,ambrym,168.12,-16.25,0.2
1,anatahan,145.67,16.35,0.2
2,aoba,167.83,-15.40,0.2
3,arenal,-84.71,10.46,0.2
4,asama,138.53,36.40,0.2


In [5]:
df_loc.loc[df_loc.lat.idxmax()]

name        hekla
long        -19.7
lat         63.98
aperture      0.3
Name: 21, dtype: object

In [6]:
## Here I am selecting Kilauea as a target
target = df_loc[df_loc.name == 'hekla'].values.flatten()

data_dir = f"data/{target[0]}/"

if not isdir(data_dir):
    mkdir(data_dir)

## Retrieving the Data for the Target
___

We have selected Kilauea as a temporary test target. Now, we will draw a box around the region and extract the data from MODVOLC.

In [7]:
from get_data import get_data

In [8]:
jyear = 2019
jday = 100
jperiod = 100000
lonmin = (target[1] - 2*target[3])
lonmax = (target[1] + 2*target[3])
latmin = (target[2] - 2*target[3])
latmax = (target[2] + 2*target[3])

df = get_data(jyear=jyear, jday=jday, jperiod=jperiod, lonmin=lonmin, lonmax=lonmax, latmin=latmin, latmax=latmax).sort_values('UNIX_Time').reset_index()
df.rename(index=str, columns={"Mo": "Month", "Dy": "Day", "Hr": "Hour", "Mn" : "Minute"}, inplace=True)

df["datetime"] = pd.to_datetime(df[["Year", "Month", "Day", "Hour", "Minute"]])
print(f"...{df.shape[0]} measurements retrieved...")

df.to_csv(f"{data_dir}data.csv")
df.head()

...951 measurements retrieved...


,index,UNIX_Time,Sat,Year,Month,Day,Hour,Minute,Longitude,Latitude,...,SunZen,SunAzi,Line,Samp,Ratio,Glint,Excess,Temp,Err,datetime
0,950,951742500,T,2000,2,28,12,55,-19.778114,63.958645,...,72.31,170.41,1987,535,-0.19,78.179,115.267,264.0,7.4,2000-02-28 12:55:00
1,937,951742500,T,2000,2,28,12,55,-19.622910,64.013390,...,72.36,170.58,1979,540,-0.43,77.830,60.840,262.4,8.7,2000-02-28 12:55:00
2,938,951742500,T,2000,2,28,12,55,-19.635347,63.969242,...,72.31,170.56,1984,541,-0.47,78.163,58.778,262.8,8.1,2000-02-28 12:55:00
3,939,951742500,T,2000,2,28,12,55,-19.641369,63.960365,...,72.31,170.55,1985,541,-0.30,78.081,84.775,262.8,8.1,2000-02-28 12:55:00
4,940,951742500,T,2000,2,28,12,55,-19.647268,63.951473,...,72.31,170.55,1986,541,-0.42,77.988,66.413,262.8,8.1,2000-02-28 12:55:00
